In [ ]:
import numpy as np
from random import random, gauss, gammavariate, seed, expovariate, choice
from statistics import stdev
from scipy.stats import chi2, norm
from math import erf, sqrt, factorial, log, exp
import matplotlib.pyplot as plt


In [ ]:
print('Ejercicio 1. De acuerdo con la teoría genética de Mendel, cierta planta')
print('de guisantes debe producir flores blancas, rosas o rojas con probabilidad')
print('1 /4, 1 /2 y 1 /4, respectivamente. Para verificar experimentalmente')
print('la teoría, se estudió una muestra de 564 guisantes, donde se encontró que')
print('141 produjeron flores blancas, 291 flores rosas y 132 flores rojas.')
print('Aproximar el pvalor de esta muestra:')
print()
print('a) utilizando un aproximación chi-cuadrada,')


def flores():
    U = random()
    if U < 0.5:
        i = 1  # rosa
    elif U < 0.75:
        i = 0  # 'blanca'
    else:
        i = 2  # roja'
    return i


probs = np.array([0.25, 0.5, 0.25])
frec_observadas = np.array([141, 291, 132])
n = sum(frec_observadas)

T = 0
for i in range(3):
    T += (frec_observadas[i] - n*probs[i])**2/(n*probs[i])

pvalor = 0
for j in range(10000):
    mtra_flores = np.zeros(3, int)  # frecuencia observada de flores simuladas
    for _ in range(n):
        mtra_flores[flores()] += 1
    t = 0
    for i in range(3):
        t += (mtra_flores[i] - n*probs[i])**2/(n*probs[i])
    if t >= T:
        pvalor += 1
pvalor /= 10000


print()
print('p valor con chi2-python: ', '{:.5f}'.format(1-chi2.cdf(T, 2)))
print()
print('b) realizando una simulación')
print()
print('p valor estimado: ', pvalor)
print()


In [ ]:
print('Ejercicio 9. En un estudio de vibraciones, una muestra aleatoria de 15 componentes del avión fueron')
print('sometidos a fuertes vibraciones hasta que se evidenciaron fallas estructurales. ')
print('Los datos proporcionados son los minutos transcurridos hasta que se evidenciaron dichas fallas.')
print('1.6 10.3 3.5 13.5 18.4 7.7 24.3 10.7 8.4 4.9 7.9 12 16.2 6.8 14.7')
print('Pruebe la hipótesis nula de que estas observaciones pueden ser consideradas')
print('como una muestra de la población exponencial')
print()
# Observación: acá se usa el pvalor estimado con la distribución empírica (exponencial), no con la uniforme
# ya que con la uniforme se obtiene una sobre estimación del p-valor


def exp_acum(x, L):
    return 1-np.exp(-L*x)


def K_S_exp(datos, L):
    n = len(datos)
    datos.sort()
    d = 0
    for j in range(n):
        x = datos[j]
        d = max(d, (j+1)/n-exp_acum(x, L), exp_acum(x, L)-j/n)
    return d


datos = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3,
         10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]

n = len(datos)
landa = n/sum(datos)

D = K_S_exp(datos, landa)

Nsim = 10000
pvalor = 0

muestra = np.empty(n)
for j in range(Nsim):
    for k in range(n):
        muestra[k] = -np.log(1-random())/landa
    d_k = K_S_exp(muestra, n/sum(muestra))
    if d_k > D:
        pvalor += 1

pvalor /= Nsim
print()


print('Estimador de Kolmogorov-Smirnov D =  ', D)
print()
print('p-valor simulado= ', pvalor)
print()
if pvalor > 0.05:
    print('No rechazar Ho; p-valor= {:.4f}, es > 0.05'.format(pvalor))
else:
    print('Rechazar Ho; p-valor= {:.4f}, es < 0.05'.format(pvalor))


# %%


In [ ]:
print('Ejercicio 11. Un experimento diseñado para comparar dos tratamientos contra la corrosión arrojó los')
print('siguientes datos (los cuales representan la máxima profundidad de los agujeros en unidades')
print('de milésima de pulgada) en pedazos de alambre sujetos a cada uno de los tratamientos por ')
print('separado:')
print('Tratamiento 1: 65.2 67.1 69.4 78.4 74.0 80.3')
print('Tratamiento 2: 59.4 72.1 68.0 66.2 58.5')
print('a) Calcular el p −valor exacto de este conjunto de datos, correspondiente a la hipótesis')
print(' de que ambos tratamientos tienen resultados idénticos.')
print('b) Calcular el p −valor aproximado en base a una aproximación normal,')
print('c) Calcular el p −valor aproximado en base a una simulación.')
print()


def norm_acum(x):
    if x >= 0:
        Fi = erf(x/sqrt(2.))/2.+0.5
    else:
        Fi = 0.5-erf(-x/sqrt(2.))/2.
    return Fi


def rangos(n, m, r):
    if n == 1 and m == 0:
        if r < 1:
            p = 0.
        else:
            p = 1.
    elif n == 0 and m == 1:
        if r < 0:
            p = 0.
        else:
            p = 1.
    else:
        if n == 0:
            p = rangos(0, m-1, r)
        elif m == 0:
            p = rangos(n-1, 0, r-n)
        else:  # n>0, m>0
            p = n/(n+m)*rangos(n-1, m, r-n-m)+m/(n+m)*rangos(n, m-1, r)
    return p


def permutacion(x):  # x=[x[0],x[1],...,x[N-1]]
    "Genera una permutación aleatoria del vector x"
    N = len(x)
    for j in range(N-1, 0, -1):
        indice = int((j+1)*random())
        x[j], x[indice] = x[indice], x[j]
    return x


trat1 = [65.2, 67.1, 69.4, 78.4, 74.0, 80.3]

control = [59.4, 72.1, 68.0, 66.2, 58.5]

n = len(trat1)
m = len(control)

trat = trat1 + control
trat.sort()
r = 0
# print(trat)
# Calculo el estadístico observado r en la muestra trat1:
for i in range(n):
    elemento = trat1[i]
    if trat.count(elemento) == 1:  # si "elemento" aparece una sola vez en la lista trat
        # "pos" guardará el índice de la posición donde aparece por primera vez "elemento" en la lista trat +1, i.e., el rando de trat1[i]
        pos = trat.index(elemento)+1
    else:
        repetido = trat.count(elemento)
        i = trat.index(elemento)+1  # primer lugar en que aparece
        # (i+(i+1) +..+(i+repetido-1)) / repetido
        pos = (i + (i + repetido-1))/2
    r += pos
    #print(elemento, pos)
print('r = ', r)
pvalor = 2*min(rangos(n, m, r), 1-rangos(n, m, r-1))  # (mirar apunte teórico)
print('pvalor con recursión:{:.4f} '.format(pvalor))  # p-valor exacto

# p-valor por simulación:
Nsim = 10000
pvalor_M = 0
pvalor_m = 0
pvalor2_M = 0
pvalor2_m = 0
for _ in range(Nsim):
    lista = []
    for k in range(n+m):
        lista.append(k+1)  # lista=[1,2,3,...]
    lista = permutacion(lista)
    # sumo los elementos de la lista permutada hasta n. Es decir, rsim tiene una simulación del rango R de trat1
    rsim = sum(lista[:n])
    rsim2 = 0
    for s in range(1, n+1):
        # rsim2 es otra simulacion de rango R de trat1
        rsim2 += lista.index(s)+1
    if rsim >= r:
        pvalor_M += 1
    if rsim <= r:
        pvalor_m += 1
    if rsim2 >= r:
        pvalor2_M += 1
    if rsim2 <= r:
        pvalor2_m += 1
pvalor = 2*min(pvalor_M, pvalor_m)/Nsim
pvalor2 = 2*min(pvalor2_M, pvalor2_m)/Nsim
print('pvalor con simulación:', pvalor, pvalor2)

# p-valor aproximando por una normal:
r_strella = (r-n*(n+m+1)/2)/sqrt(n*m*(n+m+1)/12)
pvalor = 2*min(norm_acum(r_strella), 1-norm_acum(r_strella))
print('pvalor con normales: {:.4f} '.format(pvalor))
print('rstrella: {:.4f} '.format(r_strella))


print()
# %%
print()
